# Table des Matieres

- [Rapport d'Analyse des Donnees de Pointage](#rapport-danalyse-des-donnees-de-pointage)
- [Introduction](#introduction)
- [Analyse des Absences](#analyse-des-absences)
- [Repartition Globale des Types de Journee](#repartition-globale-des-types-de-journee)
- [Analyse des Retards](#analyse-des-retards)
- [Tendance Mensuelle des Presences et Retards](#tendance-mensuelle-des-presences-et-retards)
- [Classement Complet des Employes Base sur les Retards](#classement-complet-des-employes-base-sur-les-retards)
- [Classement Complet des Employes Base sur les Absences](#classement-complet-des-employes-base-sur-les-absences)
- [Liste des Employes Pointant Apres 8h](#liste-des-employes-pointant-apres-8h)
- [Tendance Mensuelle des Absences](#tendance-mensuelle-des-absences)
- [Top 5 des Employes avec le Plus d'Absences par Direction](#top-5-des-employes-avec-le-plus-dabsences-par-direction)
- [Top 5 des Employes avec le Plus de Retards par Direction](#top-5-des-employes-avec-le-plus-de-retards-par-direction)
- [Classement Graphique des Services avec le Plus d'Absences](#classement-graphique-des-services-avec-le-plus-dabsences)
- [Classement Graphique des Directions avec le Plus d'Absences](#classement-graphique-des-directions-avec-le-plus-dabsences)
- [Analyse des Retards](#analyse-des-retards)
- [Analyse par Heure d'Entree et de Sortie](#analyse-par-heure-dentree-et-de-sortie)
- [Conclusion](#conclusion)



# Introduction

Ce notebook presente une analyse des donnees de pointage des employes, en se concentrant sur les absences et les retards par service et par direction.
L'objectif est de fournir une vue claire des services et directions les plus touches par ces problemes, avec des graphiques pour illustrer les classements.

Utilisez le menu ci-dessus pour acceder rapidement aux sections.

In [ ]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

demie_journée=6
absence=0
normal=8
# chargement des donnees 
df=pd.read_csv('../datas/data.csv')

# NETTOYAGE DES DONNEES 

# Les étapes suivantes ont été effectuées pour nettoyer les données :
# 1. Suppression des colonnes inutiles.
# 2. Nettoyage des valeurs dans la colonne 'Total présence'.
# 3. Suppression des lignes sans nom et avec 'Holiday' dans la colonne 'Horaire/Incidence'.
# 4. Traitement des colonnes d'entrée/sortie et conversion en format datetime.
# 5. Calcul du temps de présence et ajout d'une colonne 'Type'.
# Supprimer les colonnes 'Code', 'C.N.I.', 'pause' et toutes les colonnes commençant par 'Unnamed:'
colonnes_a_supprimer = ['Code', 'C.N.I.', 'pause'] + [col for col in df.columns if 'Unnamed' in col]
df.drop(columns=colonnes_a_supprimer, inplace=True)

# Remplacer les valeurs invalides par NaN et convertir en type float
df['Total présence'] = pd.to_numeric(df['Total présence'], errors='coerce')
# Suppression des lignes où 'Nom' est vide
df = df.dropna(subset=['Nom'])

# Suppression des lignes où 'Horaire/Incidence' est 'Holiday'
df = df[df['Horaire/Incidence'] != 'Holiday']
# Remplacement des valeurs manquantes dans les colonnes 'Entrée' et 'Sortie'
# Remplacement de 'Pointage manquant' par 0
df.replace({'Pointage manquant': '00:00'}, inplace=True)

# Conversion des colonnes en format datetime
cols_to_convert = ['Entrée', 'Sortie', 'Entrée.1', 'Sortie.1', 'Entrée.2', 'Sortie.2']
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_datetime, format='%H:%M', errors='coerce')

# Conversion de la colonne 'Date'
df['Date'] = pd.to_datetime(df['Date'].str.split(' ').str[0], format='%d/%m/%Y')

# Extraction des premières et dernières heures d'entrée/sortie
# Extraction des premières et dernières heures d'entrée/sortie
df['Entrée'] = df[['Entrée', 'Entrée.1', 'Entrée.2']].min(axis=1)
df['Sortie'] = df[['Sortie', 'Sortie.1', 'Sortie.2']].max(axis=1)

# Suppression des colonnes supplémentaires
df = df.drop(columns=['Entrée.1', 'Sortie.1', 'Entrée.2', 'Sortie.2'])
# Calcul du nombre d'heures passées au bureau
df['Total présence'] = (df['Sortie'] - df['Entrée']).dt.total_seconds() / 3600
df['Total présence'].fillna(0, inplace=True)  # Remplace les valeurs manquantes par 0
# Création de la colonne 'Type'
# Ajoute une colonne Type pour classifier les enregistrements en fonction du temps de présence.

# Création de la colonne 'Type'
df['Type'] = 'retard'  # Par défaut, tout est 'retard'

# Mettre à jour les types en fonction de la colonne 'Total présence'
df.loc[df['Total présence'] == absence, 'Type'] = 'absence'
df.loc[df['Total présence'] < absence, 'Type'] = 'pointage manquant'
df.loc[(df['Total présence'] >= demie_journée) & (df['Total présence'] < 8), 'Type'] = 'demie journée'
df.loc[df['Total présence'] >= normal, 'Type'] = 'normal'



In [ ]:

# chargement des donnees 
df1=pd.read_csv('../datas/data1.csv')

# NETTOYAGE DES DONNEES 

# Les étapes suivantes ont été effectuées pour nettoyer les données :
# 1. Suppression des colonnes inutiles.
# 2. Nettoyage des valeurs dans la colonne 'Total présence'.
# 3. Suppression des lignes sans nom et avec 'Holiday' dans la colonne 'Horaire/Incidence'.
# 4. Traitement des colonnes d'entrée/sortie et conversion en format datetime.
# 5. Calcul du temps de présence et ajout d'une colonne 'Type'.
# Supprimer les colonnes 'Code', 'C.N.I.', 'pause' et toutes les colonnes commençant par 'Unnamed:'
colonnes_a_supprimer = ['Code', 'C.N.I.', 'pause'] + [col for col in df1.columns if 'Unnamed' in col]
df1.drop(columns=colonnes_a_supprimer, inplace=True)

# Remplacer les valeurs invalides par NaN et convertir en type float
df1['Total présence'] = pd.to_numeric(df1['Total présence'], errors='coerce')
# Suppression des lignes où 'Nom' est vide
df1 = df1.dropna(subset=['Nom'])

# Suppression des lignes où 'Horaire/Incidence' est 'Holiday'
df1 = df1[df1['Horaire/Incidence'] != 'Holiday']
# Remplacement des valeurs manquantes dans les colonnes 'Entrée' et 'Sortie'
# Remplacement de 'Pointage manquant' par 0
df1.replace({'Pointage manquant': '00:00'}, inplace=True)

# Conversion des colonnes en format datetime
cols_to_convert = ['Entrée', 'Sortie', 'Entrée.1', 'Sortie.1', 'Entrée.2', 'Sortie.2']
df1[cols_to_convert] = df1[cols_to_convert].apply(pd.to_datetime, format='%H:%M', errors='coerce')

# Conversion de la colonne 'Date'
df1['Date'] = pd.to_datetime(df1['Date'].str.split(' ').str[0], format='%d/%m/%Y')

# Extraction des premières et dernières heures d'entrée/sortie
# Extraction des premières et dernières heures d'entrée/sortie
df1['Entrée'] = df1[['Entrée', 'Entrée.1', 'Entrée.2']].min(axis=1)
df1['Sortie'] = df1[['Sortie', 'Sortie.1', 'Sortie.2']].max(axis=1)

# Suppression des colonnes supplémentaires
df1 = df1.drop(columns=['Entrée.1', 'Sortie.1', 'Entrée.2', 'Sortie.2'])
# Calcul du nombre d'heures passées au bureau
df1['Total présence'] = (df1['Sortie'] - df1['Entrée']).dt.total_seconds() / 3600
df1['Total présence'].fillna(0, inplace=True)  # Remplace les valeurs manquantes par 0
# Création de la colonne 'Type'
# Ajoute une colonne Type pour classifier les enregistrements en fonction du temps de présence.

# Création de la colonne 'Type'
df1['Type'] = 'retard'  # Par défaut, tout est 'retard'

# Mettre à jour les types en fonction de la colonne 'Total présence'
df1.loc[df1['Total présence'] == absence, 'Type'] = 'absence'
df1.loc[df1['Total présence'] < absence, 'Type'] = 'Pointage manquant'
df1.loc[(df1['Total présence'] >= demie_journée) & (df1['Total présence'] < 8), 'Type'] = 'demie journée'
df1.loc[df1['Total présence'] >= normal, 'Type'] = 'normal'



In [ ]:
# on concatene les 2 dataset
# Concaténation horizontale
df = pd.concat([df, df1], axis=0, ignore_index=True)
# Remplacer "demie journée" par "partie tôt"
df['Type'] = df['Type'].replace('demie journée', 'partie tôt')

# Définir le type "retard" pour les personnes qui pointent après 8h
df['Type'] = np.where(df['Entrée'].dt.hour > 8, 'retard', df['Type'])

# Définir "partie tôt" pour les personnes qui quittent avant 17h
df['Type'] = np.where(df['Sortie'].dt.hour < 17, 'partie tôt', df['Type'])

noms_a_supprimer = [
    "Amadou Legrand DIOP", "Pape Waly DIOUF", "Mamadou Nana SARR", "Diegane DIAGNE",
    "El Hadji Medoune DIOUF", "Ndéye Bineta NDIAYE", "Amath SALL", "Aminata SOW", 
    "Thioro MBAYE SALL", "Madogal THIOUNE", "issa CISSE", "Yankhoba NDIAYE", 
    "Aboune DIATTA", "Yaye Nogueye KEITA", "Ibrahima DIOUF", "Louis Benoit MBAYE", 
    "Ibrahima NDIAYE", "Ndeye fatou FALL", "Mbaye DIENG", "Serigne Mansour FAYE"
]
noms_a_supprimer = [nom.lower() for nom in noms_a_supprimer]

# Convertir les noms du DataFrame en minuscules pour la comparaison
df['Nom_lower'] = df['Nom'].str.lower()

# Supprimer les lignes où les noms figurent dans la liste des noms à supprimer
df = df[~df['Nom_lower'].isin(noms_a_supprimer)]

# Réinitialiser l'index après suppression et supprimer la colonne temporaire
df.reset_index(drop=True, inplace=True)
df.drop(columns=['Nom_lower'], inplace=True)







In [ ]:
##On enregistre le dataset propre 
# df.to_csv('../datas/cleanData.csv', index=False)



# Rapport d'Analyse des Donnees de Pointage
## Introduction
Nous allons analyser un dataset contenant des informations sur la presence des employes d'une organisation. Ce df comprend 2679 enregistrements et 13 colonnes avec des informations telles que le nom de l'employe, son service, sa direction, les heures d'entree et de sortie, ainsi que la duree de presence.



In [ ]:


# ## Intervalle de Dates dans le df
start_date = df['Date'].min()
end_date = df['Date'].max()
print(f"Le dataset couvre la période du {start_date} au {end_date}.")





## Analyse des Absences

Cette section presente les resultats de l'analyse des absences par service et par direction.
Les graphiques montrent les services et les directions avec le plus grand nombre d'absences.


In [ ]:
# Supprimer les lignes où df['SERVICE:'] est égal à 'Cabinet'
df = df.loc[df['SERVICE:'] != 'Cabinet']


Cette periode nous permettra d'analyser les comportements de presence sur une semaine de travail

## Repartition Globale des Types de Journee
Cette section explore la repartition generale des types de journee, incluant les journees normales, les retards, les demi-journees, et les absences. Chaque type est represente par un graphique a barres, indiquant clairement le nombre d'employes pour chaque categorie.



In [ ]:
# ## Analyse Globale des Types de Journée
plt.figure(figsize=(10, 6))
type_counts = df['Type'].value_counts()
sns.barplot(x=type_counts.index, y=type_counts.values, palette='Blues_d')
plt.title("Répartition des Types de Journée")
plt.xlabel("Type de Journée")
plt.ylabel("Nombre d'Employés")
for i in range(len(type_counts)):
    plt.text(i, type_counts.values[i] + 50, str(type_counts.values[i]), ha='center')
plt.show()


## Analyse des Retards

Cette section presente les resultats de l'analyse des retards par service et par direction.
Les graphiques montrent les services et les directions avec le plus grand nombre de retards.


## Tendance Mensuelle des Presences et Retards

Ce graphique montre l'evolution des pointages par mois pour les differents types (presence normale, retard, absence, etc.). Cela nous permet d'identifier les mois ou il y a eu des pics d'absences ou de retards.



In [ ]:
# Group by month and calculate trends
df['Month'] = df['Date'].dt.to_period('M')

# Calculate monthly counts for 'Type' categories
monthly_trends = df.groupby(['Month', 'Type']).size().unstack(fill_value=0)

# Plotting the trends
plt.figure(figsize=(12, 7))
ax = monthly_trends.plot(kind='line', marker='o', figsize=(12, 7), linewidth=2)

# Add title and labels
plt.title('Tendance Mensuelle des Présences et Retards', fontsize=16)
plt.xlabel('Mois', fontsize=14)
plt.ylabel('Nombre d\'Enregistrements', fontsize=14)

# Add a grid for better readability
plt.grid(True)

# Annotate each point with the exact value
for line in ax.get_lines():
    for x, y in zip(line.get_xdata(), line.get_ydata()):
        label = f"{int(y)}"
        if y > 0:  # Only annotate points with values greater than 0
            ax.annotate(label, (x, y), textcoords="offset points", xytext=(0, 5), ha='center', fontsize=10)

# Show the legend outside the plot
ax.legend(title="Type", bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout to ensure the legend does not overlap
plt.tight_layout()

# Show the plot
plt.show()



## Classement Complet des Employes Base sur les Retards

Dans cette section, nous classons tous les employes en fonction du nombre de retards accumules. Ce classement affiche le nom de l'employe, sa direction, son service, ainsi que le nombre total de retards.


In [ ]:
# Filter the data for 'retard' only
retard_df = df[df['Type'] == 'retard']

# Group by employee name, direction, and service, and count occurrences of 'retard'
ranking_retard = retard_df.groupby(['Nom', 'DIRECTION:', 'SERVICE:']).size().reset_index(name='Total_Retard')

# Sort the ranking by total retards in descending order
ranking_retard_sorted = ranking_retard.sort_values(by='Total_Retard', ascending=False)

# Display the full ranking for retards
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Show the sorted ranking for retards
ranking_retard_sorted.style.hide(axis=False)


## Classement Complet des Employes Base sur les Absences

Cette section presente le classement des employes en fonction du nombre d'absences. Pour chaque employe, vous trouverez son nom, sa direction, son service, ainsi que le nombre total d'absences enregistrees.


In [ ]:
# Filter the data for 'absence' only
absence_df = df[df['Type'] == 'absence']

# Group by employee name, direction, and service, and count occurrences of 'absence'
ranking_absence = absence_df.groupby(['Nom', 'DIRECTION:', 'SERVICE:']).size().reset_index(name='Total_Absence')

# Sort the ranking by total absences in descending order
ranking_absence_sorted = ranking_absence.sort_values(by='Total_Absence', ascending=False)

# Display the full ranking for absences
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Show the sorted ranking for absences
ranking_absence_sorted.style.hide(axis=False)


Les classements ci-dessus vous montrent, dans un ordre decroissant, les employes avec le plus grand nombre de retards et d'absences. Chaque tableau presente les employes avec leur nom, leur direction, leur service, et le nombre total de retards ou d'absences.


## Liste des Employes Pointant Apres 8h

Cette section liste tous les employes ayant pointe apres 8 heures du matin. Nous calculons egalement le pourcentage d'employes qui arrivent apres cette heure, ce qui peut indiquer des habitudes de retard generalisees.


In [ ]:
# Ensure the 'Entrée' column is in datetime format, if not already
df['Entrée'] = pd.to_datetime(df['Entrée'], format='%H:%M', errors='coerce')

# Filter employees who check in after 8 AM
df['After_8'] = df['Entrée'].apply(lambda x: x.hour >= 8 if pd.notnull(x) else False)

# Group by employee name, direction, and service for those checking in after 8 AM
late_checkins = df[df['After_8']].groupby(['Nom', 'DIRECTION:', 'SERVICE:']).size().reset_index(name='Total_Checkins_After_8')

# Sort by the number of late check-ins in descending order
late_checkins_sorted = late_checkins.sort_values(by='Total_Checkins_After_8', ascending=False)

# Calculate the percentage of employees checking in after 8 AM
total_employees = df['Nom'].nunique()
total_late_employees = late_checkins_sorted['Nom'].nunique()
percentage_late = (total_late_employees / total_employees) * 100

# Display the full list and percentage

pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Display results
print(f"\nPourcentage d'employés qui se présentent après 8 heures du matin : {percentage_late:.2f}%")
print("Les employés qui se présentent après 8 heures du matin :")

late_checkins_sorted.style.hide(axis=False)



## Tendance Mensuelle des Absences

Le graphique ci-dessous montre la tendance mensuelle des absences, permettant de visualiser les mois ou les absences ont ete particulierement elevees.




In [ ]:
# Absence trend over months
absence_trends = df[df['Type'] == 'absence'].groupby('Month').size()

# Plot absence trends
plt.figure(figsize=(10, 6))
absence_trends.plot(kind='bar', color='red')
plt.title('Tendance Mensuelle des Absences')
plt.xlabel('Mois')
plt.ylabel('Nombre d\'Absences')
plt.show()

## Top 10 des Employes avec le Plus d'Absences par Direction

Dans cette analyse, nous avons classe les 10 employes ayant le plus d'absences pour chaque direction. Cela permet d'identifier les employes qui ont le plus manque de jours de travail, en se basant sur le nombre d'absences enregistrees.


In [ ]:
# ## Top 10 par Direction (Les Employés avec le Plus d'Absences)
directions = df['DIRECTION:'].unique()

for direction in directions:
    # Assuming 'Type' contains 'absence' to indicate an absence
    # Group by employee and count the number of absences
    absences_per_employee = df[(df['DIRECTION:'] == direction) & (df['Type'] == 'absence')].groupby('Nom').size()
    
    # Get the top 5 employees with the most absences
    top_5_absences = absences_per_employee.nlargest(10)
    
    # Plotting the results for most absences
    plt.figure(figsize=(10, 6))
    top_5_absences.plot(kind='bar', color='orange')
    plt.title(f"Top 10 des Employés avec le Plus d'Absences ({direction})")
    plt.xlabel("Employé")
    plt.ylabel("Nombre d'Absences")
    
    # Annotating the bars with the corresponding values
    for idx, value in enumerate(top_5_absences):
        plt.text(idx, value, int(value), ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.show()


## Top 10 des Employes avec le Plus de Retards par Direction

Dans cette analyse, nous avons classe les dix employes ayant le plus de retards pour chaque direction. Cela permet d'identifier les employes qui arrivent le plus souvent en retard au travail, en se basant sur le nombre de retards enregistres.


In [ ]:
# ## Top 10 par Direction (Les Employés avec le Plus de Retards)
directions = df['DIRECTION:'].unique()

for direction in directions:
    # Assuming 'Type' contains 'retard' to indicate lateness
    # Group by employee and count the number of retards
    retards_per_employee = df[(df['DIRECTION:'] == direction) & (df['Type'] == 'retard')].groupby('Nom').size()
    
    # Get the top 5 employees with the most retards
    top_5_retards = retards_per_employee.nlargest(10)
    
    # Plotting the results for most retards
    plt.figure(figsize=(10, 6))
    top_5_retards.plot(kind='bar', color='red')
    plt.title(f"Top 10 des Employés avec le Plus de Retards ({direction})")
    plt.xlabel("Employé")
    plt.ylabel("Nombre de Retards")
    
    # Annotating the bars with the corresponding values
    for idx, value in enumerate(top_5_retards):
        plt.text(idx, value, int(value), ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.show()


Cette courbe d'evolution montre comment la duree totale de presence des employes change chaque jour. Elle peut reveler des tendances telles que des presences plus longues ou plus courtes selon les jours de la semaine.

## Classement Graphique des Services avec le Plus d'Absences

Ce graphique montre les dix services ayant accumule le plus grand nombre d'absences. Cela permet d'identifier les services les plus touches par l'absenteisme.


In [ ]:
# ## Classement des Services et Directions (Absences)
# Grouping by service and direction, and counting absences
absences_by_service = df[df['Type'] == 'absence'].groupby('SERVICE:').size()
absences_by_direction = df[df['Type'] == 'absence'].groupby('DIRECTION:').size()

# Top 10 services with the most absences
top_10_absences_services = absences_by_service.nlargest(10)

# Plotting the top 10 services with the most absences
plt.figure(figsize=(10, 6))
top_10_absences_services.plot(kind='bar', color='orange')
plt.title("Top 10 des Services avec le Plus d'Absences")
plt.xlabel("Service")
plt.ylabel("Nombre d'Absences")

# Annotating the bars with the corresponding values
for idx, value in enumerate(top_10_absences_services):
    plt.text(idx, value, int(value), ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()




## Classement Graphique des Directions avec le Plus d'Absences

De meme, ce graphique presente les dix directions ayant accumule le plus d'absences, offrant une vue d'ensemble des directions les plus touchees par l'absenteisme.

In [ ]:
# Top 10 directions with the most absences
top_10_absences_directions = absences_by_direction.nlargest(10)

# Plotting the top 10 directions with the most absences
plt.figure(figsize=(10, 6))
top_10_absences_directions.plot(kind='bar', color='orange')
plt.title("Top 10 des Directions avec le Plus d'Absences")
plt.xlabel("Direction")
plt.ylabel("Nombre d'Absences")

# Annotating the bars with the corresponding values
for idx, value in enumerate(top_10_absences_directions):
    plt.text(idx, value, int(value), ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


## Analyse des Retards
Une analyse specifique des retards est realisee par service pour identifier les services les plus touches.


In [ ]:

# ## Analyse des Retards
# Répartition des retards par service
plt.figure(figsize=(14, 7))
sns.countplot(data=df[df['Type'] == 'retard'], x='SERVICE:')
plt.title("Répartition des Retards par Service")
plt.xlabel("Service")
plt.ylabel("Nombre de Retards")
plt.xticks(rotation=90)
plt.show()

- **Analyse des Types de Journee par Jour de la Semaine** : Cette analyse examine comment les types de journee varient en fonction du jour de la semaine.


In [ ]:
# Analyse des absences par jour de la semaine
df['Day_of_Week'] = df['Date'].dt.day_name()
plt.figure(figsize=(10, 6))
sns.countplot(data=df[df['Type'] == 'absence'], x='Day_of_Week', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
plt.title("Répartition des Absences par Jour de la Semaine")
plt.xlabel("Jour de la Semaine")
plt.ylabel("Nombre d'Absences")
plt.show()

**Temps de Presence Moyen par Jour de la Semaine** : Ce graphique montre les jours ou les employes sont les plus presents au bureau.


In [ ]:
# Analyse de la présence moyenne par jour de la semaine
avg_presence_by_day = df.groupby('Day_of_Week')['Total présence'].mean().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
plt.figure(figsize=(10, 6))
sns.barplot(x=avg_presence_by_day.index, y=avg_presence_by_day.values, palette='Pastel1')
plt.title("Temps de Présence Moyen par Jour de la Semaine")
plt.xlabel("Jour de la Semaine")
plt.ylabel("Heures Moyennes de Présence")
for i in range(len(avg_presence_by_day)):
    plt.text(i, avg_presence_by_day.values[i] + 0.1, str(round(avg_presence_by_day.values[i], 2)), ha='center')
plt.show()

In [ ]:
# Répartition des types de journée par direction
sns.countplot(data=df, x='Type', hue='DIRECTION:')
plt.title("Répartition des types de journée par direction")
plt.show()


## Analyse par Heure d'Entree et de Sortie
Explorer les heures d'entree et de sortie pour voir s'il y a des pics specifiques, ce qui pourrait indiquer des periodes de pointe pour les arrivees ou les departs.

In [ ]:
sns.histplot(df['Entrée'].dt.hour, kde=True, bins=24)
plt.title('Distribution des Heures d\'Entrée')
plt.xlabel('Heure')
plt.ylabel('Fréquence')
plt.show()

sns.histplot(df['Sortie'].dt.hour, kde=True, bins=24)
plt.title('Distribution des Heures de Sortie')
plt.xlabel('Heure')
plt.ylabel('Fréquence')
plt.show()


## Conclusion
Cette analyse detaillee du dataset de presence des employes permet de mettre en lumiere plusieurs aspects importants :

- Les types d'incidents dominants, comme les retards ou les absences.
- L'evolution quotidienne de la presence des employes, qui peut montrer des tendances utiles pour ajuster les horaires de travail.
- Les employes les plus presents globalement et au sein de chaque direction, offrant un apercu des comportements exemplaires.
- Les retards et les absences, analyses en detail pour cibler des jours ou des periodes specifiques necessitant des interventions.

Ces analyses peuvent guider des decisions manageriales eclairees, comme l'ajustement des horaires, la mise en place de mesures incitatives, ou le renforcement des politiques de ponctualite.

Les visualisations proposees permettent de rendre les donnees accessibles a un public non technique, facilitant ainsi la comprehension et l'appropriation des resultats.